# Erklärbare KI

## `Einbindung von externen Wissensbasen`

## Inhaltsverzeichnis



### Beschreibung der Methode

+ Grundlegend: Vorhandensein einer Wissensbasis
+ Erster Schritt: Wissensbasis wird während des Trainings genutzt, um das Modell zu erstellen
+ Zweiter Schritt: Vom KI-Modell getroffene Entscheidungen mit konkreten Einträgen in der Wissensbasis begründen zu können

### Allgemeines

+ Die Ausgabe der KI ist eine Entscheidung und zusätzlich eine Erklärung der Entscheidung
+ Methode ist auf alle Klassifikationsverfahren anwendbar
+ Funktioniert am besten für Klassifikationsprobleme, bei denen Texte als Inputs verwenden werden


### Vorteile
+ Leicht nachvollziehbar 
+ Kombination mehrerer Wissensbasen möglich
+ Erhöht Vertrauen
+ Hilft bei der Frage nach der  Verantwortlichkeit

### Nachteile

+ Abhängig von Qualität der Wissensbasis
+ Aufbau der Wissensbasis zeitaufwendig


### Beispiele

+ Medizinische Anwendung
    + anhand von Symptomen Krankheit vorhersagen und anschließend Vorhersage mit Publikationen unterstützen
+ Home-Automation
+ Bilderkennung


## Praktische Umsetzung
Mittels Retrieval-augmented generation (RAG) und auf Basis von [Leun23] und Llama 2

In [1]:
import pickle
from tabulate import tabulate
import pandas as pd
from studysearch.run import run as run_studysearch

## 2. Einlesen neuer Textdaten

![](assets/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

Zum Einlesen neuer Dokumente wird das Skript `build_db.py` ausgeführt. Es ließt die vorhandenen Dokumente, teil den Text in sog. chunks ein und speichert die vektorisierten Textdaten mithilfe von [JoDJ19] in eine .faiss-Datei.

In [ ]:
!python dp_build.py

## 2. Anfrage an das LLM

![](assets/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

In [ ]:
resp = run_studysearch(input="Worum geht es in dem Studiengang Digitale Technologien?")
pickle.dump(resp, open("./resp.pkl", "wb"))

### 2.2 Import einer zuvor gesicherten Antwort

In [2]:
resp = pickle.load(open("resp.pkl", "rb"))

In [3]:
resp

{'query': 'Worum geht es in dem Studiengang Digitale Technologien?',
 'result': 'The study program "Digitale Technologien (praxisintegriert)" focuses on the practical application of digital technologies in various fields, such as software development, data analytics, cybersecurity, and more. The course aims to provide students with a comprehensive understanding of the theoretical foundations of digital technologies and their practical applications in real-world scenarios.',
 'source_documents': [Document(page_content='Digitale Technologien (praxisintegriert) B.Eng.  \n9 Stellenwert der Note für die Endnote:  \ngemäß BRPO  \n10 Modulbeauftragte/r:  \nProf. Dr. rer. oec. Pascal Reusch  \n11 Sonstige Informationen:  \n \n12 Sprache:  \ndeutsch', metadata={'source': 'data/Studiengangsprüfungsordnung und Modulhandbuch Bachelor Digitale Technologien praxisintegriert Version 18.pdf', 'page': 34}),
  Document(page_content='Digitale Technologien (praxisintegriert) B.Eng.  \n9 Stellenwert der No

## 3. Ausgabe

### 3.1 Definition einer Formatierungsfunktion

Um die Ausgabe der Antwort des LLM an die Jupyter Notebook-Umgebung anzupassen, wird die Funktion `display_response(resp_dict: dict)` definiert. Diese wandelt die Antwort in Form eines `dict` in HTML-Code um.

In [3]:
def display_response(resp_dict: dict) -> str:
    """Render response in HTML using pandas and tabulate.

    Args:
        resp_dict (dict): The raw response dict eiher unpickled or from the LLM

    Returns:
        str: tabulate-generated HTML code
    """
    resp_dict = resp_dict.copy()
    resp_dict["source_documents"] = [src.metadata['source'] for src in resp_dict["source_documents"]]
    resp_df = pd.DataFrame(resp_dict).rename(columns={"query": "Query", "result": "Result", "source_documents": "Source Documents"})
    for i in range(1, len(resp_dict["source_documents"])):
        resp_df.loc[i, "Query"] = ""
        resp_df.loc[i, "Result"] = ""
    resp_df.drop_duplicates(subset=["Source Documents"], inplace=True, keep="first")
    return tabulate(resp_df, tablefmt="html", headers="keys", showindex=False)

### 3.2 Ausgabe der Antwort

Als Ergebnis erhält man neben der Antwort des LLM auch die Quell-Dokumente, die das Modell zum Finden der Antwort verwendet hat:

In [5]:
display_response(resp)

Query,Result,Source Documents
Worum geht es in dem Studiengang Digitale Technologien?,"The study program ""Digitale Technologien (praxisintegriert)"" focuses on the practical application of digital technologies in various fields, such as software development, data analytics, cybersecurity, and more. The course aims to provide students with a comprehensive understanding of the theoretical foundations of digital technologies and their practical applications in real-world scenarios.",data/Studiengangsprüfungsordnung und Modulhandbuch Bachelor Digitale Technologien praxisintegriert Version 18.pdf


## Beispiele

In [ ]:
display_response(run_studysearch(input="Worum geht es in dem Studiengang Digitale Technologien?"))

In [4]:
display_response(run_studysearch(input="Worum geht es in dem Modul Qualitätssicherung für KI-Systeme aus dem Studiengang Digitale Technologien?"))

/home/simon/Development/qki-studysearch/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Query,Result,Source Documents
Worum geht es in dem Modul Qualitätssicherung für KI-Systeme aus dem Studiengang Digitale Technologien?,"In the module Quality Assurance for AI Systems, students learn about the importance of quality assurance in the development and deployment of AI systems. They will understand how to identify and mitigate potential risks associated with AI systems, such as bias, security, and privacy concerns. Additionally, they will learn how to design and implement quality control processes for AI systems, including testing, validation, and verification methods. The module will also cover the ethical considerations of AI system development and deployment, and how to ensure that AI systems are aligned with societal values and norms.",data/Studiengangsprüfungsordnung und Modulhandbuch Bachelor Wirtschaftsingenieurwesen praxisintegriert Version 18.pdf
,,data/Studiengangsprüfungsordnung und Modulhandbuch Bachelor Mechatronik_Automatisierung praxisintegriert Version 18.pdf


In [5]:
display_response(run_studysearch(input="What is the topic of the lecture Qualitaetssicherung für KI-Systeme from the course of study Digitale Technologien?"))

Query,Result,Source Documents
What is the topic of the lecture Qualitaetssicherung für KI-Systeme from the course of study Digitale Technologien?,The topic of the lecture Qualitaetssicherung für KI-Systeme from the course of study Digitale Technologien is ensuring quality in AI systems.,data/Studiengangsprüfungsordnung für den Masterstudiengang Forschungsmaster Data Science.pdf
,,data/Studiengangsprüfungsordnung und Modulhandbuch Bachelor Angewandte Hebammenwissenschaft Version 2022.pdf


In [6]:
display_response(run_studysearch(input="When I did my Bachlor's in Digitale Technologien, which master's degree can you recommend?"))

Query,Result,Source Documents
"When I did my Bachlor's in Digitale Technologien, which master's degree can you recommend?","Based on your background in Digital Technologies, I would recommend considering the Master's program in Informatik or Wirtschaftsinformatik at Fachhochschule Bielefeld. Both of these programs align with your previous studies and can provide you with further knowledge and skills in the field of computer science and business informatics.",data/Studiengangsprüfungsordnung für den Masterstudiengang Forschungsmaster Data Science.pdf
,,data/Studiengangsprüfungsordnung für den weiterbildenden Masterstudiengang Digitale Technologien.pdf


In [ ]:
display_response(run_studysearch(input="When I did my Bachlor's in Digitale Technologien, which master's degree of other universities can you recommend?"))